# Refactored notebook for modelling

## imports

In [ ]:
import sys

sys.path.append("..")

import numpy as np
import pandas as pd
import warnings
import re
import plotly.express as px
import plotly.graph_objects as go

from NHS_PROMs.load_data import load_proms, structure_name
from NHS_PROMs.preprocess import filter_in_range, filter_in_labels, method_delta
from NHS_PROMs.utils import downcast, map_labels, fillna_categories, pd_fit_resample
from NHS_PROMs.data_dictionary import meta_dict

# use adjusted fillna which can cope with non-existing categories
pd.core.frame.DataFrame.fillna = fillna_categories
pd.core.frame.DataFrame.fillna = fillna_categories

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector
# from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.inspection import permutation_importance
from sklearn import set_config
set_config(display='diagram')

from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline, make_pipeline

# enable autodetect by using "infer" + the use of column names
SMOTENC.fit_resample = pd_fit_resample(SMOTENC.fit_resample)

## load data
General approach is not DRY for the sake of availability of having knee and hip df's always at hand, but also keep it readable (script-wise).

In [ ]:
# load data + rename columns with structired name
# df_knee_raw = load_proms(part="knee").apply(downcast).rename(structure_name, axis=1)
df_hip_raw = load_proms(part="hip").apply(downcast).rename(structure_name, axis=1)

# get meta data for each
full_meta = {t + k: v for k, v in meta_dict.items() for t in ["t0_", "t1_"]}
hip_meta = {k: v for k, v in full_meta.items() if k in df_hip_raw.columns}

df_hip_raw.sample(3)

## basic cleaning

In [ ]:
endings = (
    "code",
    "procedure",
    "revision_flag",
    "assisted_by",
    "profile",
    "predicted",
)
cols2drop = [c for c in df_hip_raw.columns if c.endswith(endings)]

In [ ]:
%%time
df_hip_clean = (
    df_hip_raw.apply(lambda s: filter_in_range(s, **hip_meta[s.name]))
    .apply(lambda s: filter_in_labels(s, **hip_meta[s.name]))
    .apply(lambda s: map_labels(s, **hip_meta[s.name]))
    .query("t0_revision_flag == 'no revision'")
    .drop(columns=cols2drop)
    .reset_index(drop=True)
    #     .replace("missing", np.nan)
)

df_hip_clean.sample(3)

## split data

In [ ]:
# split train + test set
# df_knee_seen = df_knee_clean.query("t0_year != '2019/20'")
# df_knee_unseen = df_knee_clean.query("t0_year == '2019/20'")

df_hip = df_hip_clean.query("t0_year != '2019/20'")
df_hip_unseen = df_hip_clean.query("t0_year == '2019/20'")

df_hip.sample(3)

## make feature set

In [ ]:
# asses quickly missing
print(len(df_hip), "original")
print(len(df_hip.dropna()), "after possible total dropna")
(df_hip.isna().sum() / len(df_hip)).sort_values(ascending=False).head(10)

In [ ]:
# remove NaNs from non categorical/ordinal columns (numerical)
print(len(df_hip), "original")
num_cols = df_hip.select_dtypes(exclude="category").columns
df_hip = df_hip.dropna(subset=num_cols).fillna(value="missing")

print(len(df_hip), "after dropna on numerical + fillna on categories")

In [ ]:
# create x, y
X = df_hip.filter(regex="t0")
y = df_hip["t1_ohs_score"] - df_hip["t0_ohs_score"]

# make a smaller selection of our training data to play with
X = X.iloc[:1000, -5:]
y = y.iloc[:1000]


# create train, test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## make + train a simple pipeline

In [ ]:
ct = ColumnTransformer(
    (
        ("categorical", OneHotEncoder(), make_column_selector(dtype_include="category")),
        ("numerical", StandardScaler(), make_column_selector(dtype_include="number")),
    ),
    remainder="drop",
)

pl = Pipeline(
    (
        ("by_column_types", ct),
        ("model", GradientBoostingRegressor()),
    )
)

# train the pipeline/model
pl.fit(X_train, y_train)

## predict + evaluate

In [ ]:
# # make prediction
# y_hat = pl.predict(X_test.head(500))

# # evaluate
# print(classification_report(y_test.head(500), y_hat))

## prediction intervals
Last time we were talking about confidence intervals.

But we assumed that for individual prediction we are meaning prediction intervals, correct?

## used sources
basic explaination:
* https://machinelearningmastery.com/prediction-intervals-for-machine-learning/
* https://towardsdatascience.com/quantile-regression-from-linear-models-to-trees-to-deep-learning-af3738b527c3

using parallel models with the quantile loss function for gradient boosting model:
* https://towardsdatascience.com/how-confidence-and-prediction-intervals-work-4592019576d8
* https://towardsdatascience.com/how-to-generate-prediction-intervals-with-scikit-learn-and-python-ab3899f992ed
* https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0

linear regression approach:
* https://towardsdatascience.com/prediction-intervals-in-linear-regression-2ea14d419981


Our current worked out example is based on:
* parallel gradient boosting models 
* using different quantile loss function alphas

In [ ]:
pl.named_steps["model"].set_params(loss="quantile", alpha=0.9)
pl.fit(X_train, y_train)
y_90 = pl.predict(X_test)

pl.named_steps["model"].set_params(loss="quantile", alpha=0.1)
pl.fit(X_train, y_train)
y_10 = pl.predict(X_test)

pl.named_steps["model"].set_params(loss="quantile", alpha=0.5)
pl.fit(X_train, y_train)
y_hat = pl.predict(X_test)

Plot of prediction intervals on test set

In [ ]:
pd_conf = pd.DataFrame({
    "10%": y_10,
    "90%": y_90,
    "true":y_test,
    "predicted": y_hat,
}).reset_index(drop=True)
               
# px.scatter(pd_conf)
px.scatter(pd_conf, x="true", y="predicted")

## now do it smart

In [ ]:
from joblib import Parallel
from sklearn.multioutput import MultiOutputRegressor, _fit_estimator
from sklearn.base import is_classifier
from sklearn.utils.validation import _check_fit_params
from sklearn.utils.fixes import delayed


class ConfidenceEstimator(MultiOutputRegressor):
    def __init__(self, estimator, quantiles, *, n_jobs=None):

        super().__init__(estimator, n_jobs=n_jobs)
        self.quantiles = quantiles

    def fit(self, X, y, sample_weight=None, **fit_params):
        """Fit the model to data.
        Fit a separate model for each output variable.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data.
        y : {array-like, sparse matrix} of shape (n_samples, n_outputs)
            Multi-output targets. An indicator matrix turns on multilabel
            estimation.
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.
            Only supported if the underlying regressor supports sample
            weights.
        **fit_params : dict of string -> object
            Parameters passed to the ``estimator.fit`` method of each step.
            .. versionadded:: 0.23
        Returns
        -------
        self : object
        """

        if not hasattr(self.estimator, "fit"):
            raise ValueError("The base estimator should implement" " a fit method")

        X, y = self._validate_data(
            X, y, force_all_finite=False, multi_output=False, accept_sparse=True
        )

        if is_classifier(self):
            check_classification_targets(y)

        if sample_weight is not None and not has_fit_parameter(
            self.estimator, "sample_weight"
        ):
            raise ValueError("Underlying estimator does not support" " sample weights.")

        fit_params_validated = _check_fit_params(X, fit_params)

        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_estimator)(
                self.estimator.set_params(loss="quantile", alpha=alpha),
                X,
                y,
                sample_weight,
                **fit_params_validated
            )
            for alpha in self.quantiles
        )
        return self

In [ ]:
quantiles = [0.1, 0.5, 0.9]

pl = Pipeline((
    ("by_column_types", ct),
    ("model", ConfidenceEstimator(GradientBoostingRegressor(), quantiles=quantiles)),
))

# train the pipeline/model
pl.fit(X_train, y_train)

In [ ]:
pl.predict(X_test)

In [ ]:
X_i = X_test.sample()
X_i.index[0]
y_i = y_test.loc[X_i.index[0]]
display(pl.predict(X_i), y_i)

## now do it over the top

## refactored it in a transfomer
(still parallel models)

In [ ]:
# set confidence intervals
step_size = 0.05
quantiles = np.arange(step_size, 1, step_size)

# setup pipeline
pl = Pipeline((
    ("by_column_types", ct),
    ("model", ConfidenceEstimator(GradientBoostingRegressor(), quantiles=quantiles)),
))
  
# train pipeline/model
pl.fit(X_train, y_train) 

Questions:
* Is this (in this particular form/model) what was meant in the last expert session?
* (Because we have parallel models?) sometimes strange issues?
    * eg: interval boundary 65% < 50%!
    
    How usefull is this approach then?

In [ ]:
y_int = pl.predict(X_test)
prediction_intervals = {k:v for k, v in zip(quantiles, y_int)}

def plot_prediction(intervals, true_value):
    fig = go.Figure()

    point_estimate = prediction_intervals[.5]

    for label, x in prediction_intervals.items():
        fig.add_trace(
            go.Scatter(
                x=[x, point_estimate], y=[1, 1], 
                fill='tozeroy', mode="none", 
                fillcolor='rgba(255,0,0,0.1)',
                showlegend=False,
            )
        )
        if label != .5:
            fig.add_annotation(
                x=x, y=label,
                text=f"{label*100:.0f}%",
                showarrow=False,
            )

    fig.add_trace(
        go.Scatter(
            x=[point_estimate]*2, y=[0, 1],
            mode="lines", line={"color":"red"}, 
            name="point estimate",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[true_value]*2, y=[0, 1],
            mode="lines", line={"color":"blue"}, 
            name="true value",
        )
    )

    x = list(prediction_intervals.values())
    x_range = [np.min(x), np.max(x)]
    x_range = (x_range - np.mean(x_range)) * 1.1 + np.mean(x_range)
    fig.update_xaxes(range=x_range)
    fig.update_yaxes(visible=False, showticklabels=False)
    fig.update_layout(height=400)

    fig.show()

In [ ]:
# take one sample from test set
X_i = X_test.sample()
y_i = y_test.loc[X_i.index[0]]
# predict including prediction intervals
y_int = pl.predict(X_i)[0]

# plot prediction intervals
prediction_intervals = {k:v for k, v in zip(quantiles, y_int)}
plot_prediction(intervals=prediction_intervals, true_value=y_i)

In [ ]:
# import plotly.graph_objs as go
# fig = go.Figure([
#     go.Scatter(
#         name='Prediction',
#         x=,
#         y=df['10 Min Sampled Avg'],
#         mode='lines',
#         line=dict(color='rgb(31, 119, 180)'),
#     ),
#     go.Scatter(
#         name='Upper Bound',
#         x=df['Time'],
#         y=df['10 Min Sampled Avg']+df['10 Min Std Dev'],
#         mode='lines',
#         marker=dict(color="#444"),
#         line=dict(width=0),
#         showlegend=False
#     ),
#     go.Scatter(
#         name='Lower Bound',
#         x=df['Time'],
#         y=df['10 Min Sampled Avg']-df['10 Min Std Dev'],
#         marker=dict(color="#444"),
#         line=dict(width=0),
#         mode='lines',
#         fillcolor='rgba(68, 68, 68, 0.3)',
#         fill='tonexty',
#         showlegend=False
#     )
# ])
# fig.update_layout(
#     yaxis_title='Wind speed (m/s)',
#     title='Continuous, variable value error bars',
#     hovermode="x"
# )
# fig.show()

## Gridsearch

In [ ]:
# create parameter grid to search on 
# standard (same as pipeline)
param_grid = dict()

# construct gridsearch
GS = GridSearchCV(pl, param_grid=param_grid, scoring="f1") ## add scoring

# train gridsearch
GS.fit(X_train, y_train)

# show results
pd.DataFrame(GS.cv_results_)\
    .filter(regex=r"^(?!.*(split|time)).*$")\
    .set_index("rank_test_score").sort_index()

In [ ]:
# create parameter grid to search on 

# standard same as pipeline
param_grid = dict()

# # # two models with default parameters
param_grid = {"model": [KNeighborsClassifier(), DecisionTreeClassifier()]}

# # tuning hyper parameters
param_grid = {
    "model": [RandomForestClassifier(), AdaBoostClassifier()],
    "model__n_estimators": [25, 50, 100],
}

# tuning different hyper parameters on different models
param_grid = [
    {
        "model": [RandomForestClassifier()],
        "model__n_estimators": [25, 50, 100],
    },
    {
        "model": [KNeighborsClassifier()],
        "model__n_neighbors": [2, 5, 10],
    },
]


# # construct gridsearch

# # standard
# GS = GridSearchCV(pl, param_grid=param_grid)

# # # # add scoring 
# GS = GridSearchCV(pl, param_grid=param_grid, scoring="f1")

# # multiple
GS = GridSearchCV(pl, param_grid=param_grid, scoring=["balanced_accuracy", "f1"], refit=False)


# train gridsearch
GS.fit(X_train, y_train)

# show results
pd.DataFrame(GS.cv_results_)\
    .filter(regex=r"^(?!.*(split|time)).*$")\
#     .set_index("rank_test_score").sort_index()

## regression

In [ ]:
# create x, y
X = df_hip.filter(regex="t0")
y = df_hip["t1_ohs_score"] - df_hip["t0_ohs_score"]


# create train, test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# make a smaller selection of our training data to play with
X_train = X_train.iloc[:1000, -5:]
y_train = y_train.iloc[:1000]

In [ ]:
# make parameter grid
param_grid = {
    "balancer": ["passthrough"],
    "model": [KNeighborsRegressor()],
}

GS = GridSearchCV(pl, param_grid=param_grid)
# train gridsearch
GS.fit(X_train, y_train)

# show results
pd.DataFrame(GS.cv_results_)\
    .filter(regex=r"^(?!.*(split|time)).*$")
#     .set_index("rank_test_score").sort_index()

## extract feature names pl

In [ ]:
# get the feature names from pipeline
def get_feature_names(sklobj, feature_names=[]):

    if isinstance(sklobj, Pipeline):
        for name, step in sklobj.steps:
            get_feature_names(step, feature_names)
    elif isinstance(sklobj, ColumnTransformer):
        for name, transformer, columns in sklobj.transformers_:
            feature_names += get_feature_names(transformer, columns)
    elif isinstance(sklobj, OneHotEncoder):
        feature_names = sklobj.get_feature_names(feature_names).tolist()
    elif isinstance(sklobj, str):
        if sklobj == "passthrough":
            pass
        elif sklobj == "drop":
            feature_names = []
            
    return feature_names

In [ ]:
get_feature_names(pl)

In [ ]:
# # this is slow ...
# r = permutation_importance(pl, X_train.head(1_000), y_train.head(1_000), n_repeats=2, random_state=0)

# feature_names = get_feature_names(pl)

# for i in r.importances_mean.argsort()[::-1]:
#     if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#         print(f"{feature_names[i]:<8}"
#         f"{r.importances_mean[i]:.3f}"
#         f" +/- {r.importances_std[i]:.3f}")

## a more advanced pipeline

In [ ]:
# TO DO ...